<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Cleaning</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Establishing-rules" data-toc-modified-id="Establishing-rules-1.0.0.1"><span class="toc-item-num">1.0.0.1&nbsp;&nbsp;</span>Establishing rules</a></span></li><li><span><a href="#Filling-the-blank" data-toc-modified-id="Filling-the-blank-1.0.0.2"><span class="toc-item-num">1.0.0.2&nbsp;&nbsp;</span>Filling the blank</a></span></li></ul></li><li><span><a href="#Imports-and-overview" data-toc-modified-id="Imports-and-overview-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Imports and overview</a></span></li><li><span><a href="#Rules-for-data-cleaning" data-toc-modified-id="Rules-for-data-cleaning-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Rules for data cleaning</a></span></li><li><span><a href="#Further-improvement-include-leveraging-Levenshtein-distance" data-toc-modified-id="Further-improvement-include-leveraging-Levenshtein-distance-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Further improvement include leveraging Levenshtein distance</a></span></li></ul></li></ul></li></ul></div>

# Data Cleaning

#### Establishing rules

We need to establish rules for a consistent cleaning.

#### Filling the blank

We need to decide which strategy to adopt when dealing with missing data

### Imports and overview

In [226]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from typing import List

engine = create_engine('sqlite:///../data/data.db', echo=False)
con = engine.connect()
df = pd.read_sql('select * from patient', con=con)
con.close()

In [22]:
df.head()

,patient_id,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
0,221958,matisse,clarke,13.0,rene street,ellenbrook,2527,wa,19710708.0,32.0,08 86018809,westella
1,771155,joshua,elrick,23.0,andrea place,east preston,2074,nsw,19120921.0,34.0,02 97793152,foxdown
2,231932,alice,conboy,35.0,mountain circuit,prospect,2305,nsw,19810905.0,22.0,02 20403934,None
3,465838,sienna,craswell,39.0,cumberlegeicrescent,henty,3620,wa,19840809.0,30.0,02 62832318,jodane
4,359178,joshua,bastiaans,144.0,lowrie street,campbell town,4051,nsw,19340430.0,31.0,03 69359594,None


### Rules for data cleaning

- First remove patient with duplicated ids
- Then remove patients with null names
- Then remove patients with duplicated phone number
- Then remove patients with duplicated associated names and postcode
- Then remove patients with duplicated associated names and address_1

In [234]:
def detect_duplicates(df:pd.DataFrame) -> pd.DataFrame:
    # remove the patients with duplicated id
    df = df[~df.patient_id.duplicated()]
    # remove the patients with null names
    df['name'] = df['given_name'] + ' ' + df['surname']
    df = df[~df.name.isnull()]
    
    
    # Using phone number to detect duplicates
    # first remove the rows with phone numbers being None
    df = df.dropna(subset=["phone_number"])
    # then remove patient ids corresponding to duplicated phone numbers
    df = df[~df.patient_id.isin(set(df[df.phone_number.duplicated()].patient_id.values))]
    
    def rm_gb(cols:List[str], data:pd.DataFrame) -> pd.DataFrame:
        '''groups by columns and removes rows where groupby count resulted in more than 1 example
        
        if there are more than 1 occurences of "name" == "Gerard" and "state"=="nsw" then 
        rm_gb(['name', 'state']) => should remove it
        '''
        gb_cols = data.groupby(cols).count()
        gb_cols = gb_cols[gb_cols.patient_id > 1].reset_index()
        dup = data[data[cols[0]].isin(gb_cols[cols[0]])]
        for col in cols[1:]:
            dup = dup[df[col].isin(gb_cols[col])]
        data = data[~data.patient_id.isin(dup[dup.name.duplicated()].patient_id.values)]
        return data
    
    df = rm_gb(['name', 'postcode'], df)
    df = rm_gb(['name', 'address_1'], df)
    df = rm_gb(['name', 'suburb'], df)
    return df

df = detect_duplicates(df)

c:\users\marc\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Further improvement include leveraging Levenshtein distance